# Model 
## (en consola se esta corriendo 4 periodos sin reblogeo, solo dará como resultado el integrado)
## acá 1 periodo con reblogeo

In [2]:
import pandas as pd
from integratedModel import IntegratedModel
from undergroundModel import UndergroundModel
from openPitModel import OpenPitModel

In [34]:
class Main:
    def __init__(self, path, undergroundDatabaseName, openPitDatabaseName):
        self.path = path
        self.openPitDatabaseName = openPitDatabaseName
        self.undergroundDatabaseName = undergroundDatabaseName
        self.runIntegratedModel = False
        self.runOpenPitModel = False
        self.runUndergroundModel = True
        self.numberOfPeriods = 15
        self.models = []
        
    def execute(self):
        print("Setting MineDatabases")
        self.setMineDatabases()

        #plotResults(self.variableValues, self.CA_blocks, self.openPitBlocksLenghtLimits, self.openPitBlocksWidthLimits,
        #self.openPitBlocksHeightLimits, self.undergroundBlocksWidthLimits, self.undergroundBlocksHeightLimits)
        #print(self.variableValues)
        #self.getResults(self.model)

    def setMineDatabases(self):
        if self.runIntegratedModel:  
            self.openMineDataframe = pd.read_excel(self.path+self.openPitDatabaseName, engine="openpyxl")
            self.undergroundMineDataframe = pd.read_excel(self.path+self.undergroundDatabaseName, engine="openpyxl")
        else:
            if self.runUndergroundModel:
                self.undergroundMineDataframe = pd.read_excel(self.path+self.undergroundDatabaseName, engine="openpyxl")

            if self.runOpenPitModel:
                self.openMineDataframe = pd.read_excel(self.path+self.openPitDatabaseName, engine="openpyxl")
    
    def createModels(self):
        if self.runIntegratedModel:  
            self.createIntegratedModel()

        if self.runUndergroundModel:
            self.createUndergroundModel()

        if self.runOpenPitModel:
            self.createOpenPitModel()

    def createIntegratedModel(self):
        self.integratedModel = IntegratedModel(self.undergroundMineDataframe, self.openMineDataframe,self.numberOfPeriods)
        self.models.append(self.integratedModel)

    def createUndergroundModel(self):
        self.undergroundModel = UndergroundModel(self.undergroundMineDataframe, self.numberOfPeriods)
        self.models.append(self.undergroundModel)

    def createOpenPitModel(self):
        self.openPitModel = OpenPitModel(self.openMineDataframe, self.numberOfPeriods)
        self.models.append(self.openPitModel)

    def getResults(self):
        for model in self.models:
            if isinstance(model, UndergroundModel):
                self.undergroundObjValue, self.undergroundVariableValues, self.undergroundRuntime, self.undergroundGap = model.execute()
                print("Underground model")
                print("Objective Value: {} RunTime: {} GAP: {}" .format(self.undergroundObjValue, self.undergroundRuntime, self.undergroundGap))
                self.calculateUndergroundCapacitiesPerPeriod()
            if isinstance(model, OpenPitModel):
                self.openPitObjValue, self.openPitVariableValues, self.openPitRuntime, self.openPitGap = model.execute()
                print("Open Pit model")
                print("Objective Value: {} RunTime: {} GAP: {}" .format(self.openPitObjValue, self.openPitRuntime, self.openPitGap))
                self.calculateOpenPitCapacitiesPerPeriod()
            if isinstance(model, IntegratedModel):
                self.integratedObjValue, self.integratedVariableValues, self.integratedRuntime, self.integratedGap = model.execute()
                print("Integrated model")
                print("Objective Value: {} RunTime: {} GAP: {}" .format(self.integratedObjValue, self.integratedRuntime, self.integratedGap))
            

    def calculateOpenPitCapacitiesPerPeriod(self):
        self.openPitTonelagePerPeriod = {}
        self.openPitMineralPerPeriod = {}
        for period in range(self.numberOfPeriods):
            self.openPitTonelagePerPeriod[period] = 0
            self.openPitMineralPerPeriod[period] = 0
            for block in range(len(self.openPitVariableValues)//(self.numberOfPeriods+1)-1):
                self.openPitTonelagePerPeriod[period] += self.openPitVariableValues[period*len(self.openPitVariableValues)//(self.numberOfPeriods+1)+block] * self.openPitModel.openPitBlockTonnage[block]
                self.openPitMineralPerPeriod[period] += self.openPitVariableValues[period*len(self.openPitVariableValues)//(self.numberOfPeriods+1)+block] * self.openPitModel.openPitBlockMineral[block]
        print("Open pit mineral per period", self.openPitMineralPerPeriod)
        print("Open pit tonelage per period", self.openPitTonelagePerPeriod)

    def calculateUndergroundCapacitiesPerPeriod(self):
        variablesAggruped = [self.undergroundVariableValues[x:x+self.numberOfPeriods] for x in range(self.numberOfPeriods*len(self.undergroundModel.drawpoint), 2*self.numberOfPeriods*len(self.undergroundModel.drawpoint), self.numberOfPeriods)]
        self.undergroundTonelagePerPeriod = {}
        self.undergroundMineralPerPeriod = {}
        for period in range(self.numberOfPeriods):
            self.undergroundTonelagePerPeriod[period] = 0
            self.undergroundMineralPerPeriod[period] = 0
            for variable in range(len(variablesAggruped)):
                self.undergroundTonelagePerPeriod[period] += variablesAggruped[variable][period] * self.undergroundModel.TON_d[variable]
                self.undergroundMineralPerPeriod[period] += variablesAggruped[variable][period] * self.undergroundModel.MIN_D[variable]
        print("Underground mineral per period", self.undergroundMineralPerPeriod)
        print("Underground tonelage per period", self.undergroundTonelagePerPeriod)

In [35]:
path = "C:/Users/Williams Medina/Desktop/Tesis Magister/Magister/ThesisCode/MainCode/Databases/integratedModel/"
undergroundDatabaseName = 'Modelo_F_OG.xlsx'
#openPitDatabaseName = 'Modelo_F_OG.xlsx'
openPitDatabaseName = 'Modelo_F_OG_4_4_4.xlsx'
main = Main(path, undergroundDatabaseName, openPitDatabaseName)
main.execute()


Setting MineDatabases


In [36]:
main.createModels() 

In [37]:
main.getResults()

Set parameter TimeLimit to value 3600
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 14190 rows, 8100 columns and 140162 nonzeros
Model fingerprint: 0x378d0423
Model has 180 quadratic constraints
Variable types: 2700 continuous, 5400 integer (5400 binary)
Coefficient statistics:
  Matrix range     [3e-01, 5e+05]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [3e+04, 1e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 2e+31]
  QRHS range       [8e-01, 8e-01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 2851 rows and 135 columns
Presolve time: 0.11s
Presolved: 16857 rows, 10635 columns, 80031 nonzeros
Variable types: 5340 continuous, 5295 integer (5295 binary)
Found heuristic solution: objective 1.147911e+09
Found heuristic solution: objective 1.

In [6]:
main.undergroundRuntime

0.006977081298828125

In [7]:
main.openPitRuntime

827.9644794464111

In [8]:
main.integratedRuntime

23.183870315551758

In [10]:
main.calculateUndergroundCapacitiesPerPeriod()

IndexError: list index out of range

In [11]:
main.undergroundVariableValues

[1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0

In [31]:
variablesAggruped = [main.undergroundVariableValues[x:x+15] for x in range(15*len(main.undergroundModel.drawpoint), 30*len(main.undergroundModel.drawpoint), 15)]
main.undergroundTonelagePerPeriod = {}
main.undergroundMineralPerPeriod = {}
for period in range(main.numberOfPeriods):
    main.undergroundTonelagePerPeriod[period] = 0
    main.undergroundMineralPerPeriod[period] = 0
    for variable in range(len(variablesAggruped)):
        print(variable, period)
        print(variablesAggruped[variable][period])
        print(main.undergroundModel.TON_d[variable])
        print(main.undergroundTonelagePerPeriod[period])
        main.undergroundTonelagePerPeriod[period] += variablesAggruped[variable][period] * main.undergroundModel.TON_d[variable]
        main.undergroundMineralPerPeriod[period] += variablesAggruped[variable][period] * main.undergroundModel.MIN_D[variable]
print("Underground mineral per period", main.undergroundMineralPerPeriod)
print("Underground tonelage per period", main.undergroundTonelagePerPeriod)

0 0
0.7
486000
0
1 0
0.7
486000
340200.0
2 0
0.7000000000000005
486000
680400.0
3 0
0.7000000000000005
486000
1020600.0000000002
4 0
0.7000000000000005
486000
1360800.0000000005
5 0
0.7000000000000005
486000
1701000.0000000007
6 0
0.7000000000000005
486000
2041200.000000001
7 0
0.7000000000000005
486000
2381400.000000001
8 0
0.7000000000000005
486000
2721600.000000001
9 0
0.7000000000000005
486000
3061800.000000001
10 0
0.7000000000000005
486000
3402000.000000001
11 0
0.7000000000000005
486000
3742200.000000001
12 0
0.7000000000000005
486000
4082400.000000001
13 0
0.7000000000000005
486000
4422600.000000001
14 0
0.3
486000
4762800.000000001
15 0
0.7000000000000005
486000
4908600.000000001
16 0
0.7000000000000005
486000
5248800.000000001
17 0
0.7000000000000005
486000
5589000.000000001
18 0
0.7000000000000005
486000
5929200.000000001
19 0
0.7000000000000005
486000
6269400.000000001
20 0
0.7000000000000005
486000
6609600.000000001
21 0
0.7000000000000005
486000
6949800.000000001
22 0
0.7

In [23]:
len(variablesAggruped)

180

In [29]:
variablesAggruped

[[1.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0],
 [1.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0],
 [1.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0],
 [1.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0],
 [1.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0],
 [1.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0],
 [1.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0],
 [1.0, 0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0],
 [1.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0],
 [1.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0],
 [1.0, -0.0, -0.0, -0

In [30]:
main.undergroundVariableValues

[1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0

In [42]:
a = sum(main.undergroundTonelagePerPeriod.values())

In [43]:
a

87188400.0